In [ ]:
import os
import sys

notebook_dir = os.getcwd()
project_root_path = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root_path)

from src import ModelXtoC
from scripts.run_preprocessing import preprocessing_main
from src.utils import *

In [2]:
concept_labels, train_loader, val_loader, test_loader = preprocessing_main(verbose=True)

Found 11788 images.
Processing in 369 batches of size 32 (for progress reporting)...


Processing batches: 100%|█████████████████████| 369/369 [00:55<00:00,  6.61it/s]



Finished processing.
Successfully transformed: 11788 images.
Found 11788 unique images.
Found 312 unique concepts.
Generated concept matrix with shape: (11788, 312)
Found 200 classes.
Found labels for 11788 images.
Generated one-hot matrix with shape: (11788, 200)
Split complete: 5994 train images, 5794 test images.
Dataset initialized with 5994 pre-sorted items.
Dataset initialized with 5794 pre-sorted items.


# TRAINING IMPLEMENTATION (IN PROGRESS)

In [3]:
from src.utils import find_class_imbalance
from src import inception_v3
from config import N_CONCEPTS, N_CLASSES
import torch
import torch.nn as nn
import torch.optim as optim
import time

In [4]:
device = torch.device("cuda" if torch.cuda.is_available()
                    else "mps" if torch.backends.mps.is_available()
                    else "cpu")
print(f"Using device: {device}")

Using device: mps


In [5]:
# Instantiate the model
model = ModelXtoC(pretrained=True,
                freeze=True,
                n_classes=N_CLASSES, # Still needed for model structure, but won't be trained/used
                use_aux=True,
                n_concepts=N_CONCEPTS)

model = model.to(device)
print("Model Instantiated (X -> C)")

Model Instantiated (X -> C)


### Loss
Original code from CBM Github uses a separate loss for each attribute

`BCEWithLogitsLoss()` performs 2 steps:
1. $\sigma(x)$
    - Applies the sigmoid function to the logits to get probabilities.
2. $\text{BCE}(\sigma(x), y) = y \cdot \text{log}(\sigma(x)) + (1-y) \cdot (1-\text{log}(\sigma(x)))$
    - Compute binary cross-entropy between output probabilities ($\sigma(x)$) and ground truths ($y$)

In [6]:
use_weighted_loss = True # Set to False for simple unweighted loss

if use_weighted_loss:
    concept_weights = find_class_imbalance(concept_labels)
    attr_criterion = [nn.BCEWithLogitsLoss(weight=torch.tensor([ratio], device=device, dtype=torch.float))
                    for ratio in concept_weights]
else:
    attr_criterion = [nn.BCEWithLogitsLoss() for _ in range(N_CONCEPTS)]

### Optimiser

In [7]:
lr = 0.01
weight_decay = 0.00004
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                    lr=lr,
                    momentum=0.9,
                    weight_decay=weight_decay)

scheduler_step = 1000 # Set large for almost constant LR, or smaller (e.g., 30, 50) for decay
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=0.1)

print("Optimizer and Scheduler Ready")

Optimizer and Scheduler Ready


### HELPERS

### Training and Validation Loops

In [8]:
# --- Training Loop ---
from src.training import run_epoch_x_to_c

epochs = 50 # Adjust as needed
log_interval = 50 # How often to print progress

# print("\nStarting Training Loop...")
best_val_acc = 0.0

for epoch in range(epochs):
    print(f"--- Epoch {epoch+1}/{epochs} ---")

    # Train
    train_loss, train_acc = run_epoch_x_to_c(model, train_loader, attr_criterion, optimizer, is_training=True, use_aux=True, n_concepts=N_CONCEPTS, device=device, log_interval=log_interval)
    print(f'Epoch {epoch+1} Train Summary | Loss: {train_loss:.4f} | Acc: {train_acc:.3f}')

    # Validate
    if test_loader: # Assuming test_loader is your validation loader
        with torch.no_grad():
            val_loss, val_acc = run_epoch_x_to_c(model, val_loader, attr_criterion, optimizer, is_training=False, use_aux=False, n_concepts=N_CONCEPTS, device=device, log_interval=log_interval)
        print(f'Epoch {epoch+1} Val Summary   | Loss: {val_loss:.4f} | Acc: {val_acc:.3f}')

#         # Save best model based on validation accuracy
#         if val_acc > best_val_acc:
#             print(f"Validation accuracy improved ({best_val_acc:.3f} -> {val_acc:.3f}). Saving model...")
#             best_val_acc = val_acc
#             torch.save(model.state_dict(), 'x_to_c_best_model.pth')
#             print("Model saved to x_to_c_best_model.pth")

    # Scheduler step
    scheduler.step()
    print(f"Current LR: {optimizer.param_groups[0]['lr']}")

print("\nTraining Finished.")
# Load best model for potential further use/testing
model.load_state_dict(torch.load('x_to_c_best_model.pth'))
print("Best model loaded.")

--- Epoch 1/50 ---
 Batch: 50/149 | Avg. Loss: 12.6132 | Acc: 86.639 (79.935) | Time: 64.12s
 Batch: 100/149 | Avg. Loss: 10.3432 | Acc: 87.430 (83.558) | Time: 39.07s
Epoch 1 Train Summary | Loss: 9.3374 | Acc: 84.884
Epoch 1 Val Summary   | Loss: 4.6143 | Acc: 89.454
Current LR: 0.01
--- Epoch 2/50 ---
 Batch: 50/149 | Avg. Loss: 6.6800 | Acc: 87.059 (87.749) | Time: 44.21s
 Batch: 100/149 | Avg. Loss: 6.6678 | Acc: 87.440 (87.735) | Time: 39.46s
Epoch 2 Train Summary | Loss: 6.5408 | Acc: 87.735
Epoch 2 Val Summary   | Loss: 4.2268 | Acc: 89.561
Current LR: 0.01
--- Epoch 3/50 ---
 Batch: 50/149 | Avg. Loss: 6.2529 | Acc: 88.732 (87.785) | Time: 43.94s
 Batch: 100/149 | Avg. Loss: 6.2263 | Acc: 87.740 (87.826) | Time: 39.50s
Epoch 3 Train Summary | Loss: 6.2265 | Acc: 87.850
Epoch 3 Val Summary   | Loss: 4.1397 | Acc: 89.664
Current LR: 0.01
--- Epoch 4/50 ---
 Batch: 50/149 | Avg. Loss: 6.2427 | Acc: 88.492 (88.015) | Time: 44.12s
 Batch: 100/149 | Avg. Loss: 6.1054 | Acc: 87.320 (

KeyboardInterrupt: 